In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    access_token = userdata.get('DEFORMER_TOKEN')
    !pip install git+https://$access_token@github.com/ay94/deformer-extractor.git@error-handling

In [2]:
from experiment_utils.configurations import (
    TrainingConfig,
    TokenizationConfig,
    ModelConfig
)

from experiment_utils.train import (
    DatasetManager,
    ModelManager,
    FineTuneUtils,
    Trainer
)




In [3]:
from experiment_utils import colab
from experiment_utils.general_utils import FileHandler

2024-07-20 07:48:06 - INFO - PyTorch version 1.13.1 available.


In [4]:
from experiment_utils import colab
from experiment_utils.general_utils import FileHandler

base_folder = colab.init('My Drive')
corpora_path = base_folder / 'Final Year Experiments/Thesis-Experiments/ExperimentData'
corpora_fh = FileHandler(corpora_path)
corpora  = corpora_fh.load_json('corpora.json')

config = base_folder / 'Final Year Experiments/Thesis-Experiments/scripts'
config_fh = FileHandler(config)
config = config_fh.load_yaml('config.yaml')


2024-07-20 07:48:06 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Data-Extraction-Phase/notebooks/My Drive
2024-07-20 07:48:06 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com


In [5]:
finetune_config = TrainingConfig.from_dict(config.get('finetuning'))
tokenization_config = TokenizationConfig.from_dict(config.get('tokenization'))
model_config = ModelConfig.from_dict(config.get('model'))
data_manager = DatasetManager(corpora, 'ANERCorp_CamelLab', tokenization_config)
num_tags = len(data_manager.corpus['labels'])
model_manager = ModelManager(num_tags, model_config)



2024-07-20 07:48:06 - INFO - Configuration validated successfully


In [6]:
trainer = Trainer(data_manager, model_manager, finetune_config)

2024-07-20 07:48:06 - INFO - Loading BERT Model from: aubmindlab/bert-base-arabertv02
Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2024-07-20 07:48:10 - I

In [7]:
metrics = trainer.training_loop()

2024-07-20 07:48:12 - INFO - Standard
2024-07-20 07:48:12 - INFO - Start Training Epoch: 1


Training:   0%|          | 0/260 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Training 

In [3]:

base_folder = colab.init('My Drive')
config_path = base_folder / 'Final Year Experiments/Thesis-Experiments/scripts'
data_path = base_folder / 'Final Year Experiments/Thesis-Experiments/ExperimentData'
fh = FileHandler(config_path)
data_fh = FileHandler(data_path)

2024-07-19 16:45:48 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Data-Extraction-Phase/notebooks/My Drive
2024-07-19 16:45:48 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com


In [4]:

from experiment_utils.train import FineTuneConfig, TCDataset
from experiment_utils.tokenization import TokenStrategyFactory


# Example usage
config_values = fh.load_yaml('train_config.yaml')
tokenization_config = fh.load_yaml('tokenization_config.yaml')
config = FineTuneConfig()


ImportError: cannot import name 'FineTuneConfig' from 'experiment_utils.train' (/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Data-Extraction-Phase/deformer-extractor/experiment_utils/train.py)

In [ ]:
data  = data_fh.load_json('corpora.json')
NERData = data['ANERCorp_CamelLab']

In [ ]:
dataset[1]['input_ids'][:36]

In [ ]:
dataset[1]['labels'][:36]

In [ ]:
model = TCModel(num_tag=10, path='bert-base-uncased')


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, BertForTokenClassification, BertConfig
from torch.nn.utils.rnn import pad_sequence

# Define a custom dataset
class NERDataset(Dataset):
    def __init__(self, tokenizer, texts, tags, label_map):
        self.tokenizer = tokenizer
        self.texts = texts
        self.tags = tags
        self.label_map = label_map

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.tokenizer.encode_plus(
            self.texts[idx],
            max_length=128,
            truncation=True,
            padding='max_length',
            return_tensors="pt",
            is_split_into_words=True
        )
        labels = [self.label_map[tag] for tag in self.tags[idx]]
        labels = torch.tensor(labels + [self.label_map['O']] * (128 - len(labels)))  # Pad labels
        return tokens['input_ids'].squeeze(0), tokens['attention_mask'].squeeze(0), labels

# Dummy data
texts = [
    ["Hello", "world", "this", "is", "BERT"],
    ["Another", "sentence"]
]
tags = [
    ["O", "O", "O", "O", "B-PER"],
    ["O", "O"]
]
label_map = {"O": 0, "B-PER": 1, "I-PER": 2}

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Initialize dataset
dataset = NERDataset(tokenizer, texts, tags, label_map)
loader = DataLoader(dataset, batch_size=2)


In [ ]:
input_ids.shape[1]

In [ ]:
# Initial model output without hidden states and attentions
for batch in loader:
    input_ids, attention_mask, labels = batch
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)


In [ ]:
model.enable_hidden_states()

In [ ]:
# Initial model output without hidden states and attentions
for batch in loader:
    input_ids, attention_mask, labels = batch
    token_type_ids = torch.tensor([0] * input_ids.shape[1], dtype=torch.long)
    outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)


In [ ]:
torch.save(model.state_dict(), 'model_state_dict.pt')


In [ ]:
loaded_model = TCModel(num_tag=10, path='bert-base-uncased')

loaded_model.load_state_dict(torch.load('model_state_dict.pt'))

In [ ]:
# Initial model output without hidden states and attentions
for batch in loader:
    input_ids, attention_mask, labels = batch
    token_type_ids = torch.tensor([0] * input_ids.shape[1], dtype=torch.long)
    outputs = loaded_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)


In [ ]:
loaded_model.enable_attentions()

In [ ]:
# Initial model output without hidden states and attentions
for batch in loader:
    input_ids, attention_mask, labels = batch
    token_type_ids = torch.tensor([0] * input_ids.shape[1], dtype=torch.long)
    outputs = loaded_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)


In [ ]:
outputs.keys()

# Results handling

In [4]:
base_folder = colab.init('My Drive')
config_path = base_folder / 'Final Year Experiments/Class Imbalance/1_fineTuning'
fh = FileHandler(config_path)

2024-07-19 16:45:54 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Data-Extraction-Phase/notebooks/My Drive
2024-07-19 16:45:54 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com


In [5]:
model_name='arabertv02'
data_name='ANERCorp_CamelLab'

In [6]:
import torch
from tqdm.autonotebook import tqdm
model_outputs = fh.load_pickle(
            f"evalOutputs/{model_name}_{data_name}_regular_outputs.pkl"
        )

# load_model_path = fh.file_path / f"trainOutputs/{model_name}_{data_name}_regular.bin"
# model = torch.load(load_model_path, map_location=torch.device('cpu'))

In [9]:
import torch
data_loader = model_outputs.val_dataloader

# Assuming 'data_loader' is your DataLoader instance
# and that it has an attribute 'dataset' which stores the tensors

data = [(data_loader.dataset[i]) for i in range(len(data_loader.dataset))]

# Save to disk
torch.save(data, 'val.pth')


In [14]:
loaded_data[0]

{'input_ids': tensor([    2, 40508, 32479,    19, 33014, 34675, 43752,   466,    19,  2279,
         25896,  1089,  1098,  1161,   921,  7410, 16661,   306,  2767,  4119,
         19020, 31490,  8500, 10926, 18694,  1000, 13567,   139,  2801,  3011,
          7996,   305,  7377, 40508, 50014,   197,   305, 26258,  2050,   492,
          3812,  6700, 18112,   306,  9520,  2657, 20195,    20,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [21]:
# Load the data
loaded_data = torch.load('test.pth')
from torch.utils.data import DataLoader, TensorDataset



# Create a new DataLoader
loaded_data = torch.load('test.pth')
new_data_loader = DataLoader(loaded_data, batch_size=8)



In [ ]:
data_loader = model_outputs.test_dataloader
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [ ]:
model.eval()
with torch.no_grad():
	final_loss = 0
	preds = None
	labels = None
	for data in tqdm(data_loader, total=len(data_loader)):
		for k, v in data.items():
			data[k] = v.to(device)
		outputs = model(**data)
		loss = outputs["average_loss"]
		logits = outputs["logits"]
		final_loss += loss.item()
		if logits is not None:
			preds = (
				logits if preds is None else torch.cat((preds, logits), dim=0)
			)
		if data["labels"] is not None:
			labels = (
				data["labels"]
				if labels is None
				else torch.cat((labels, data["labels"]), dim=0)
			)
	preds = preds.detach().cpu().numpy()
	labels = labels.cpu().numpy()
	# evaluation = Evaluation(labels, preds, inv_labels, final_loss)
	# metrics = evaluation.compute_metrics()

In [ ]:
from experiment_utils.train import Evaluation, Metrics

evaluator = Evaluation(model_outputs.data['inv_labels'], labels, preds, final_loss)
results = evaluator.generate_results()
metrics = Metrics.from_dict(results)


In [ ]:
outputs.test_metrics.skl_metrics

# Trainer

In [3]:
import logging
from dataclasses import dataclass, field
from typing import Optional, Any, Dict


@dataclass
class FineTuneConfig:
    train_batch_size: int
    test_batch_size: int
    shuffle: bool
    epochs: int
    splits: int
    learning_rate: float
    warmup_ratio: float
    max_grad_norm: float
    accumulation_steps: int
    logging_step: int

    def __post_init__(self):
        self.validate_config()
        
    @staticmethod
    def from_dict(config_dict: Dict[str, Any]):
        try:
            config_dict['learning_rate'] = float(config_dict['learning_rate'])
        except ValueError as e:
            logging.error("Invalid format for learning_rate, needs to be convertible to float")
            raise ValueError("Invalid format for learning_rate") from e
        return FineTuneConfig(**config_dict)
    
    def validate_config(self):
        if not (0 < self.learning_rate < 1):
            logging.error("Invalid learning rate: %s", self.learning_rate)
            raise ValueError("Learning rate must be between 0 and 1")
        if not (0 < self.warmup_ratio < 1):
            logging.error("Invalid warmup ratio: %s", self.warmup_ratio)
            raise ValueError("Warmup ratio must be between 0 and 1")
        if self.epochs <= 0:
            logging.error("Invalid number of epochs: %s", self.epochs)
            raise ValueError("Epochs must be greater than 0")
        if self.train_batch_size <= 0 or self.test_batch_size <= 0:
            logging.error("Invalid batch sizes: Train %s, Valid %s", self.train_batch_size, self.test_batch_size)
            raise ValueError("Batch sizes must be greater than 0")
        if self.accumulation_steps < 1:
            logging.error("Invalid accumulation steps: %s", self.accumulation_steps)
            raise ValueError("Accumulation steps must be at least 1")
        if self.logging_step < 1:
            logging.error("Invalid logging steps: %s", self.logging_step)
            raise ValueError("Accumulation steps must be at least 1")
        logging.info("Configuration validated successfully")



@dataclass
class TokenizationStrategy:
    type: str
    index: int
    schema: Optional[str] = None

@dataclass
class TokenizationConfig:
    tokenizer_path: str
    preprocessor_path: str
    max_seq_len: int
    strategy: TokenizationStrategy
    
    def __post_init__(self):
        self.validate_config()
        
    @staticmethod
    def from_dict(config_dict: Dict[str, Any]):
        # This assumes the 'strategy' sub-dictionary is properly formatted
        strategy_config = TokenizationStrategy(**config_dict['strategy'])
        return TokenizationConfig(
            tokenizer_path=config_dict['tokenizer_path'],
            preprocessor_path=config_dict['preprocessor_path'],
            max_seq_len=config_dict['max_seq_len'],
            strategy=strategy_config
        )

    def validate_config(self):
        if not isinstance(self.max_seq_len, int) or self.max_seq_len <= 0:
            raise ValueError("max_seq_len must be a positive integer")
        if not self.tokenizer_path:
            raise ValueError("tokenizer_path cannot be empty")
        if not self.preprocessor_path:
            raise ValueError("preprocessor_path cannot be empty")
        if not isinstance(self.strategy, TokenizationStrategy):
            raise ValueError("strategy must be an instance of TokenizationStrategy")
        if self.strategy.type not in ['core', 'all']:  # Example check
            raise ValueError("Invalid strategy type specified")
        if not isinstance(self.strategy.index, int) or self.strategy.index < 0:
            raise ValueError("Strategy index must be a non-negative integer")



@dataclass
class ModelConfig:
    model_path: str
    dropout_rate: float
    enable_attentions: bool
    enable_hidden_states: bool
    initialize_output_layer: bool

    def __post_init__(self):
        self.validate_config()

    @staticmethod
    def from_dict(config_dict):
        return ModelConfig(**config_dict)

    def validate_config(self):
        if not self.model_path:
            raise ValueError("Model path cannot be empty")
        if not (0 <= self.dropout_rate <= 1):
            raise ValueError("Dropout rate must be between 0 and 1")
        if not isinstance(self.enable_attentions, bool):
            raise ValueError("enable_attentions must be a boolean value")
        if not isinstance(self.enable_hidden_states, bool):
            raise ValueError("enable_hidden_states must be a boolean value")
        if not isinstance(self.initialize_output_layer, bool):
            raise ValueError("initialize_output_layer must be a boolean value")



@dataclass
class TrainingArguments:
    train_batch_size: int
    test_batch_size: int
    max_grad_norm: int
    accumulation_steps: int
    epochs: int
    splits: int
    shuffle: bool
    # Including splits here depends on whether you treat 'splits' as a training process configuration
    # or part of a dataset configuration. If 'splits' refers to cross-validation or similar, it might belong here.

    @staticmethod
    def from_config(config):
        # Assuming 'config' is a dictionary containing these keys
        return TrainingArguments(
            train_batch_size=config.train_batch_size,
            test_batch_size=config.test_batch_size,
            max_grad_norm=config.max_grad_norm,
            accumulation_steps=config.accumulation_steps,
            epochs=config.epochs,
            splits=config.epochs,
            shuffle=config.shuffle
        )

    def validate_arguments(self):
        if self.train_batch_size <= 0 or self.test_batch_size <= 0:
            raise ValueError("Batch sizes must be greater than 0")
        if self.epochs <= 0:
            raise ValueError("Number of epochs must be positive")
        if self.epochs <= 0:
            logging.info("Shuffle Argument is %s", self.shuffle)



In [4]:




class DatasetManager:
    def __init__(self, corpora, dataset_name, config, config_dict):
        self.corpus = self.get_corpus(dataset_name, corpora)
        self.config = config
        self.config_dict = config_dict
    
    def get_corpus(self, data_name, corpora):
        if data_name not in corpora:
            raise ValueError(f"Data name {data_name} not found in corpora.")
        return corpora[data_name]
    
    def get_dataset(self, split):
        self.data = self.corpus['splits']
        return self.create_dataset(split)

    def create_dataset(self, split):
        return TCDataset(
            texts=[x['words'] for x in self.data[split]],
            tags=[x['tags'] for x in self.data[split]],
            label_map=self.corpus["labels_map"],
            config=self.config,
            config_dict=self.config_dict
        )
    
    
    def get_dataloader(self, split, batch_size, shuffle=False):
        try:
            return torch.utils.data.DataLoader(
                dataset=self.create_dataset(split),
                batch_size=batch_size,
                shuffle=shuffle,
                num_workers=2
            )
        except:
            logging.error("The %s Split Doesn't Exist", split)
        return None
    
        

import logging
import torch
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from experiment_utils.train import TCModel, TCDataset, FineTuneUtils







class ModelManager:
    def __init__(self, num_tags, config):
        self.num_tags = num_tags
        self.config = config
                
    def configure_model(self):
        device = self.get_device()
        model = TCModel(self.num_tags, self.config)
        model.to(device)
        self.original_state = model.state_dict()
        logging.info("Model %s loaded and sent to %s", self.config.model_path, device)
        return model
    
    def get_device(self):
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def get_model_parameters(self):
        model = self.configure_model()
        return model.named_parameters()
    
    def reset_model(self):
        self.model.load_state_dict(self.original_state)  # Reset to initial state
        return model.to(self.device) 

    # def save_model(self, save_path):
    #     torch.save(self.model.state_dict(), save_path)
    #     logging.info("Model saved to %s", save_path)

    # def load_model(self, model_path):
    #     self.model.load_state_dict(torch.load(model_path, map_location=self.device))
    #     logging.info("Checkpoint loaded from %s", model_path)

class OptimizerSchedulerManager:
    def __init__(self, model_parameters, num_train_examples, config):
        self.model_parameters = model_parameters
        self.num_train_examples = num_train_examples
        self.config = config
        
    def configure_optimizer_and_scheduler(self):
        logging.info("Configuring the optimizer with learning rate: %s", self.config.learning_rate)
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.model_parameters if not any(nd in n for nd in no_decay)],
             'weight_decay': 0.01},
            {'params': [p for n, p in self.model_parameters if any(nd in n for nd in no_decay)],
             'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.config.learning_rate)
        scheduler = self.configure_scheduler(optimizer)
        logging.info("Optimizer configured successfully")
        return optimizer, scheduler

    def configure_scheduler(self, optimizer):
        steps_per_epoch = (self.num_train_examples// (self.config.train_batch_size * self.config.accumulation_steps))
        total_steps = steps_per_epoch * self.config.epochs
        logging.info("Configuring scheduler with total steps: %d", total_steps)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=int(total_steps * self.config.warmup_ratio),  # 10% of training steps for warmup
            num_training_steps=total_steps
        )
        logging.info("Scheduler configured successfully")
        return scheduler
    
    def reset_optimizer_and_scheduler(self):
        return self.configure_optimizer_and_scheduler()

In [5]:
from experiment_utils import colab
from experiment_utils.general_utils import FileHandler

base_folder = colab.init('My Drive')
corpora_path = base_folder / 'Final Year Experiments/Thesis-Experiments/ExperimentData'
corpora_fh = FileHandler(corpora_path)
corpora  = corpora_fh.load_json('corpora.json')

config = base_folder / 'Final Year Experiments/Thesis-Experiments/scripts'
config_fh = FileHandler(config)
config = config_fh.load_yaml('config.yaml')


2024-07-19 00:00:45 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Data-Extraction-Phase/notebooks/My Drive
2024-07-19 00:00:45 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com


In [6]:
finetune_config = FineTuneConfig.from_dict(config.get('finetuning'))
tokenization_config = TokenizationConfig.from_dict(config.get('tokenization'))
model_config = ModelConfig.from_dict(config.get('model'))

2024-07-19 00:00:46 - INFO - Configuration validated successfully


In [8]:
# TODO: Change the way the strategy factory works as currently it is restiricted to joson
data_manager = DatasetManager(corpora, 'ANERCorp_CamelLab', tokenization_config, config.get('tokenization'))
num_tags = len(data_manager.corpus['labels'])
model_manager = ModelManager(num_tags, model_config)
optimizer_scheduler_manager = OptimizerSchedulerManager(model_manager.get_model_parameters(), len(data_manager.get_dataset('train')), finetune_config)
args = TrainingArguments.from_config(finetune_config)


2024-07-19 00:00:49 - INFO - Loading BERT Model from: aubmindlab/bert-base-arabertv02
Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2024-07-19 00:00:53 - I

In [38]:
import time

class Trainer:
    def __init__(self, finetune_utils, data_manager, model_manager, optimizer_scheduler_manager, args) -> None:
        self.model = None
        self.device = None
        self.optimizer = None
        self.scheduler = None
        self.train_dataloader = None
        self.test_dataloader = None
        self.validation_dataloader = None
        self.finetune_utils = finetune_utils
        self.args = args
        self.setup_trainer(data_manager, model_manager, optimizer_scheduler_manager)
        
    def setup_trainer(self, data_manager, model_manager, optimizer_scheduler_manager):
        self.model = model_manager.configure_model()
        self.device = model_manager.get_device()
        self.optimizer, self.scheduler = optimizer_scheduler_manager.configure_optimizer_and_scheduler()
        # self.train_dataloader = data_manager.get_dataloader('train', self.args.train_batch_size)
        self.train_dataloader = model_outputs.train_dataloader
        # self.test_dataloader = data_manager.get_dataloader('test', self.args.test_batch_size)
        self.test_dataloader = model_outputs.test_dataloader
        # self.validation_dataloader = data_manager.get_dataloader('validation', self.args.test_batch_size)
        
    def train(self):
        training_loss = self.fine_tune.train_fn(
            self.train_dataloader,
            self.model,
            self.optimizer,
            self.device,
            self.scheduler,
            self.args,
        )
        return training_loss
    def evaluate(self, dataloader):
        eval_metrics, eval_loss = self.fine_tune.eval_fn(
            dataloader, self.model, self.device, self.data_manager.corpus.get("inv_map")
        )
        return eval_metrics, eval_loss
    def training_loop(self):
        for epoch in range(self.args.epochs):
            logging.info("Start Training Epoch: %d", epoch + 1)
            start_time = time.time()
            training_loss = self.train()

            logging.info("Start Evaluation")
            eval_metrics, eval_loss = self.evaluate(self.validation_dataloader if self.validation_dataloader else self.test_dataloader)  

            logging.info("Training Loss: %f | Eval Loss: %f", training_loss, eval_loss)
            elapsed_time = time.time() - start_time
            if elapsed_time < 3600:
                logging.info("Epoch completed in %.2f minutes", elapsed_time / 60)
            else:
                logging.info("Epoch completed in %.2f hours", elapsed_time / 3600)
            print(eval_metrics)
    def cross_validation(self):
        pass

In [39]:
model_outputs.train_dataloader

In [40]:
trainer = Trainer(FineTuneUtils, data_manager, model_manager, optimizer_scheduler_manager, args)

2024-07-19 00:17:28 - INFO - Loading BERT Model from: aubmindlab/bert-base-arabertv02
Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2024-07-19 00:17:33 - I

In [29]:
args

TrainingArguments(train_batch_size=16, test_batch_size=8, max_grad_norm=1.0, accumulation_steps=1, epochs=4, splits=4, shuffle=False)